In [54]:
import random
import data
import os
import parameters as p
import tensorflow as tf
import importlib
import viewer as v

In [55]:
importlib.reload(p)
importlib.reload(data)

<module 'data' from 'C:\\Users\\user\\PycharmProjects\\nuty5\\jupyter\\data.py'>

In [56]:
random.seed()
classes = data.retrieve_class_names()
output_dim = p.GX * p.GY * (5 + len(classes))
pages = data.Retriever().retrieve(classes)

27% of the files processed, we are at 500 in 1864.
54% of the files processed, we are at 1000 in 1864.
80% of the files processed, we are at 1500 in 1864.


In [57]:
files = os.listdir(p.PNG_PATH)
generator = data.Generator(pages, output_dim, classes)
ds = tf.data.Dataset.from_generator(
    generator.generator,
    args=[files],
    output_types=(tf.int32, tf.float32),
    output_shapes=([p.BATCH_SIZE, p.Y, p.X, 3], [p.BATCH_SIZE, output_dim]))

In [58]:
from keras import backend as K

def get_loss(y_actual,y_pred):

    y_actual_reshaped = K.reshape(y_actual, [p.BATCH_SIZE * p.GX * p.GY, -1])
    y_pred_reshaped = K.reshape(y_pred, [p.BATCH_SIZE * p.GX * p.GY, -1])

    main_probs_act = y_actual_reshaped[..., 0:1]
    main_probs_pred = y_pred_reshaped[..., 0:1]
    bb_probs_act = y_actual_reshaped[..., 1:5]
    bb_probs_pred = y_pred_reshaped[..., 1:5 ]
    classes_probs_act = y_actual_reshaped[..., 5:]
    classes_probs_pred =  y_pred_reshaped[..., 5:]

    #loss = K.sum(K.binary_crossentropy(main_probs_act, main_probs_pred, from_logits=True))
    loss = K.sum(K.abs(main_probs_act - main_probs_pred))
    loss += K.sum(K.sum(K.abs(bb_probs_act - bb_probs_pred), 1) * main_probs_act)
    loss += K.sum(K.sum(K.abs(classes_probs_act - classes_probs_pred), 1) * main_probs_act)

    return loss


In [72]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(p.Y,p.X, 3))

model = tf.keras.applications.efficientnet.EfficientNetB7(
    include_top=False, weights='imagenet',
    input_tensor=inputs
)

x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(output_dim, activation="relu", name="pred")(x)
model = tf.keras.Model(inputs, outputs)

# model = tf.keras.Sequential(vgg.layers)
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(output_dim, activation='relu'))

In [ ]:
importlib.reload(p)
importlib.reload(data)
model.compile(optimizer='adam',
              loss=get_loss,
              metrics=tf.keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None))
a = model. fit(ds, epochs=1)

      5/Unknown - 126s 15s/step - loss: 95287.0781 - mean_absolute_error: 0.6375

In [ ]:
aa = generator.generator(files).__next__()
aaa = model.predict(aa[0])

v.show_prediction(aaa, aa[0])